# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [1]:
#Dando imports:
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import emoji
from emoji import UNICODE_EMOJI

In [7]:
#Leitura dos arquivos em Excel
TweetsStarbucks = pd.read_excel('Tweets_Starbucks.xlsx')

#Pegando somente o tweet:
TweetsStarbucksTreinamento = TweetsStarbucks.drop('Classificação', axis=1)

#Função para separar os EMOJIS:
Lista_EMOJI = []
for e in TweetsStarbucksTreinamento["Treinamento"]:
    vazio = ''
    for palavra in e:
        if palavra in UNICODE_EMOJI:
            vazio = vazio + " "+ palavra + " "
        else:
            vazio+= palavra
    Lista_EMOJI.append(vazio)
    
TweetsStarbucks["Treinamento"] = Lista_EMOJI


#função para limpar os caracteres, os links e as risadasdos dos tweets :
def limpeza(tweet):
    
    tweetlista = str(tweet)
    tweetlista = tweetlista.replace(",","")
    tweetlista = tweetlista.replace("#","")
    tweetlista = tweetlista.replace("@","")
    tweetlista = tweetlista.replace("!","")
    tweetlista = tweetlista.replace("$","")
    tweetlista = tweetlista.replace("%","")
    tweetlista = tweetlista.replace("&","")
    tweetlista = tweetlista.replace(".","")
    tweetlista = tweetlista.replace(";","")
    tweetlista = tweetlista.replace("¨","")
    tweetlista = tweetlista.replace(":","")
    tweetlista = tweetlista.replace("_","")
    tweetlista = tweetlista.replace("","")
    tweetlista = tweetlista.replace("-","")
    tweetlista = tweetlista.replace("rt","")
    tweetlista = tweetlista.replace("?","")
    tweetlista = tweetlista.replace(">","")
    tweetlista = tweetlista.replace("<","")
    tweetlista = tweetlista.replace("»","")
    tweetlista = tweetlista.replace("…","")
    tweetlista = tweetlista.replace("1","")
    tweetlista = tweetlista.replace("2","")
    tweetlista = tweetlista.replace("3","")
    tweetlista = tweetlista.replace("4","")
    tweetlista = tweetlista.replace("5","")
    tweetlista = tweetlista.replace("6","")
    tweetlista = tweetlista.replace("7","")
    tweetlista = tweetlista.replace("8","")
    tweetlista = tweetlista.replace("9","")
    tweetlista = tweetlista.replace("(","")
    tweetlista = tweetlista.replace(")","")
    tweetlista = tweetlista.replace("[","")
    tweetlista = tweetlista.replace("]","")
    tweetlista = tweetlista.replace("+","")
    
        
    tweetlimpo = tweetlista
    tweetlimpo = tweetlimpo.split()
    
    for palavra in tweetlimpo:
        if 'https' in palavra:
            tweetlimpo.remove(palavra)
    for Palavra in tweetlimpo:
        if 'kk' in Palavra:
            tweetlimpo.remove(Palavra) 
    for Palavra in tweetlimpo:
        if 'hah' in Palavra:
            tweetlimpo.remove(Palavra)
    for Palavra in tweetlimpo:
        if 'rsr' in Palavra:
            tweetlimpo.remove(Palavra)
 
    return tweetlimpo  



# CRIANDO UMA LISTA COM VÁRIAS LISTAS ONDE CADA UMA TEM UM TWEET
tweetslimpolista = []
for e in TweetsStarbucks.Treinamento:
    tweetslimpos= limpeza(e)
    tweetslimpolista.append(tweetslimpos)


In [8]:
#Vamos juntar as palavras dos tweets relevantes e muito relevantes em um dicionario para facilitar na contagem e separação


dictrel = {}

for e in TweetsStarbucks.Treinamento[TweetsStarbucks.Classificação == 3]:
        tweetslimpos = limpeza(e)
        for i in tweetslimpos:
            if i in dictrel:
                dictrel[i]+=1
            else:
                dictrel[i]=1


for e in TweetsStarbucks.Treinamento[TweetsStarbucks.Classificação == 4]:
        tweetslimpos = limpeza(e)
        for i in tweetslimpos:
            
            if i in dictrel:
                dictrel[i]+=1
            else:
                dictrel[i]=1

#contador de palavras relevantes
soma_relevante =  0
for palavra in dictrel:
    soma_relevante += dictrel[palavra]
    

soma_relevante                


#Vamos juntar as palavras dos tweets irrelevantes e muito irrelevantes em um dicionario para facilitar na contagem e separação

dictirrel = {}

for e in TweetsStarbucks.Treinamento[TweetsStarbucks.Classificação == 1]:
        tweetslimpos = limpeza(e)
        for i in tweetslimpos:
            if i in dictirrel:
                dictirrel[i]+=1
            else:
                dictirrel[i]=1



for e in TweetsStarbucks.Treinamento[TweetsStarbucks.Classificação == 2]:
        tweetslimpos = limpeza(e)
        for i in tweetslimpos:
            
            if i in dictirrel:
                dictirrel[i]+=1
            else:
                dictirrel[i]=1



#contador de palavras irrelevantes
soma_irrelevante =  0
for palavra in dictirrel:
    soma_irrelevante += dictirrel[palavra]
    
    
    
#total de palavras:
total_de_palavras = soma_relevante + soma_irrelevante
total_de_palavras


5491

In [9]:
#CALCULANDO A PROBABILIDADE DE SER IRRELEVANTE E RELEVANTE DADO OS TWEETS

Numero_tweets1=TweetsStarbucks.Treinamento[TweetsStarbucks.Classificação == 1]
Numero_tweets2=TweetsStarbucks.Treinamento[TweetsStarbucks.Classificação == 2]
p_irrel=(len(Numero_tweets1)+len(Numero_tweets2))/300
Numero_tweets3=TweetsStarbucks.Treinamento[TweetsStarbucks.Classificação == 3]
Numero_tweets4=TweetsStarbucks.Treinamento[TweetsStarbucks.Classificação == 4]
p_rel=(len(Numero_tweets3)+len(Numero_tweets4))/300

print(p_irrel)    # probabilidade de ser irrelevante
print(p_rel)      # probabilidade de ser relevante

0.27
0.73


In [10]:
#Função para calcular a probabilidade de ser relevante ou irrelevante

def probabilidade(lista_de_tweets):
    
    prob_relevante = p_rel
    prob_irrelevante = p_irrel
    lista_classificacao = []
    lista_classificacao_irr = []
    
    for tweet in lista_de_tweets:
        
        for palavra in tweet:
            
            pir_total = p_irrel
            pr_total = p_rel
            
            if palavra not in dictrel:
                x = 1/(soma_relevante+total_de_palavras)
                pr_total += math.log(x)
                
            if palavra in dictrel:
                x = dictrel[palavra]/(soma_relevante+total_de_palavras)
                pr_total += math.log(x)

            if palavra not in dictirrel:
                x = 1/(soma_irrelevante+total_de_palavras)
                pir_total += math.log(x)
                
            if palavra in dictirrel:
                x = dictirrel[palavra]/(soma_irrelevante+total_de_palavras)
                pir_total += math.log(x)
                
        if pr_total > pir_total:
             lista_classificacao.append(tweet)
                
        elif pr_total < pir_total:
             lista_classificacao_irr.append(tweet)
                
    return ("P(relevante) = {0}  P(irrelevante) = {1}".format((len(lista_classificacao))/(len(lista_de_tweets)),(len(lista_classificacao_irr))/(len(lista_de_tweets))))


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [11]:
TweetsStarbucksteste = pd.read_excel("Tweets_Starbucks.xlsx",sheet_name = 1)

#limpeza da parte de teste:

Lista_EMOJI = []
for e in TweetsStarbucksteste["Teste"]:
    vazio = ''
    for palavra in e:
        if palavra in UNICODE_EMOJI:
            vazio = vazio + " "+ palavra + " "
        else:
            vazio+= palavra
    Lista_EMOJI.append(vazio)
    
TweetsStarbucksteste["Teste"] = Lista_EMOJI

#aplicando o código de calculo da probabilidade criado no treinamento
lista_teste = []
for frase in TweetsStarbucksteste.Teste:
    tweetslimposteste = limpeza(frase)
    lista_teste.append(tweetslimposteste)
print(probabilidade(lista_teste))

# Com o código limpo podemos rodar para calcular a probabilidade de cada modo de classificação

contador0 = TweetsStarbucksteste.Classificação[TweetsStarbucksteste.Classificação == 1].count()
contador1 = TweetsStarbucksteste.Classificação[TweetsStarbucksteste.Classificação == 2].count()
contador2 = TweetsStarbucksteste.Classificação[TweetsStarbucksteste.Classificação == 3].count()
contador3 = TweetsStarbucksteste.Classificação[TweetsStarbucksteste.Classificação == 4].count()

contadorTotal = TweetsStarbucksteste.Classificação.count()
contador01 = contador0+contador1


P_01 = (contador0+contador1)/contadorTotal 
P_02 = (contador2+contador3)/contadorTotal 
P_0 = contador0/contadorTotal
P_1 = contador1/contadorTotal 
P_2 = contador2/contadorTotal
P_3 = contador3/contadorTotal
P_4 = P_01
P_5 = P_02

print("Probabilidade de ser irrelevante: {:.2f}%".format(P_4*100))
print("Probabilidade de ser relevante: {:.2f}%".format(P_5*100))
print("Probabilidade de ser muito irrelevante: {:.2f}%".format(P_0*100))
print("Probabilidade de ser pouco irrelevante: {:.2f}%".format(P_1*100))
print("Probabilidade de ser pouco relevante: {:.2f}%".format(P_2*100))
print("Probabilidade de ser muito relevante: {:.2f}%".format(P_3*100))

P(relevante) = 0.935  P(irrelevante) = 0.065
Probabilidade de ser irrelevante: 26.50%
Probabilidade de ser relevante: 73.50%
Probabilidade de ser muito irrelevante: 9.00%
Probabilidade de ser pouco irrelevante: 17.50%
Probabilidade de ser pouco relevante: 65.00%
Probabilidade de ser muito relevante: 8.50%


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


   A partir do classificador de Naive Bayes, é possível concluir que o método utilizado é uma ferramenta extremamente útil para a segregação e agrupamento de termos semelhantes/não semelhantes. A precisão de **78%** agrada bastante os engenheiros e(ou) desenvolvedores deste classificador, já que a separação dos tweets, neste caso especifico (em seguida falaremos de outras possíveis aplicações) é feita por relevância que é comprovadamente eficaz. Dentre as inúmeras possíveis aplicações para o classificador, podem ser citadas: um filtro de spam para e-mails; ferramenta de auxílio a diagnósticos médicos; categorizar notícias de jornais e com potencial para ser utilizado em programas de reconhecimento facial!
Nós, desenvolvedores deste classificador, imaginamos que: analisar a enorme base de dados disponíveis, é o ponto de partida para as companhias modernas consigam elevar sua posição em relação a sua concorrência. Nosso programa através de Machine Learning relaciona as opiniões das pessoas sobre determinado assunto ou produto e a relevância que o mesmo possui sob o olhar do consumidor.

## Proxima implementação

Uma sugestão para uma futura implementação: reside no fato de que a inteligência artificial tem dificuldade em detectar gírias, ironias e sarcasmos por exemplo. Uma alternativa para tentar solucionar essas barreiras é uma maior coleta e analise de dados com a integração da biblioteca “scikit-learn”.

*matrial de suporte:*
http://scikit-learn.org/stable/